In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import sklearn

## 데이터 로드 및 train test로 분할

In [19]:
df = pd.read_csv('38_Oasis.csv').drop(columns = ['Unnamed: 0']).reset_index().drop(columns=['index','link','target'])

In [20]:
df.head()

,title,date,content
0,[제목 : [오아시스]\n 팬데믹시대에 내마음의 표준을 바꾸는 유통 ],2020-09-04,이번 코로나가 유통시장의 판도를 5년이상 앞당겨진 변화를 가져 왔다?매장에서 인테넷...
1,[제목 : [오아시스] 오아시스 매수 했습니다. ],2021-04-17,"오아이스 주식 중 pe가 보유한 물량이 시장에서 움직이는 듯 하네요.어제, 아시는..."
2,[제목 : [오아시스] 오아시스 매수 했습니다. ],2021-04-17,얼마에 사셨어요? 저도 관심이 있는데요.
3,[제목 : [오아시스]\n 오아시스 주식이요 ],2021-04-17,오아시스 주식을 사고싶어게시판에 올려봅니다파실분 계시면**** 팔육일팔 삼육사일로연...
4,[제목 : [오아시스] 70만원이면 비싼건가요 ],2021-04-19,"30주 70만원에 살려고하는데, 비싼건가요"


## null 값 확인

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    136 non-null    object
 1   date     136 non-null    object
 2   content  133 non-null    object
dtypes: object(3)
memory usage: 3.3+ KB


In [22]:
# NULL 값 있는 행 확인
df[df['content'].isnull()]

,title,date,content
31,[제목 : [오아시스] 상장노력은 하는거죠? ],2021-07-09,NaN
40,[제목 : [오아시스] 조용히 갑시다~~ ],2021-07-16,NaN
43,[제목 : [오아시스] 지금 잘가고 있긴 한데 스피드 좀 올려~~ ],2021-07-29,NaN


In [23]:
# 공백으로 NULL 값 채우기
df = df.fillna('')

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    136 non-null    object
 1   date     136 non-null    object
 2   content  136 non-null    object
dtypes: object(3)
memory usage: 3.3+ KB


내용에 빈 값이 있지만 title에서 추출할 것이 있지 않을까 해서 공백으로 채워줌

## index 날짜로 바꿔주기

In [25]:
# date를 오름차순 정렬
df.date = df.date.sort_index(ascending=True)
# index로 설정
df = df.set_index('date')
df

,title,content
date,,
2020-09-04,[제목 : [오아시스]\n 팬데믹시대에 내마음의 표준을 바꾸는 유통 ],이번 코로나가 유통시장의 판도를 5년이상 앞당겨진 변화를 가져 왔다?매장에서 인테넷...
2021-04-17,[제목 : [오아시스] 오아시스 매수 했습니다. ],"오아이스 주식 중 pe가 보유한 물량이 시장에서 움직이는 듯 하네요.어제, 아시는..."
2021-04-17,[제목 : [오아시스] 오아시스 매수 했습니다. ],얼마에 사셨어요? 저도 관심이 있는데요.
2021-04-17,[제목 : [오아시스]\n 오아시스 주식이요 ],오아시스 주식을 사고싶어게시판에 올려봅니다파실분 계시면**** 팔육일팔 삼육사일로연...
2021-04-19,[제목 : [오아시스] 70만원이면 비싼건가요 ],"30주 70만원에 살려고하는데, 비싼건가요"
...,...,...
2022-08-31,[제목 : [오아시스] 우크라이나와 러시아 ],덩치 큰 러시아가 누구다 다 이길거라고 했지만 결과는? 길고 짧은건 대봐야 안다고우...
2022-08-31,"[제목 : [오아시스] 오늘 날짜(2022.08.31) 더벨 기사, 오아시스 기업...",코스닥 상장을 추진 중인 신선식품 새벽배송 기업 오아시스가 기업가치 목표를 최대 2...
2022-09-02,[제목 : [오아시스] 이 시기에 오히려 주목해야 할 기업 ],연일 금리가 급등하고 있다. 매출만을 키우기 위해 무한확장을 거듭했던 스타트업과 유...


## 특수문자 제거 정규표현식

In [26]:
df['title'] = df['title'].str.replace(r"[^\uAC00-\uD7A30-9a-zA-Z\s]",'')
df['content'] = df['content'].str.replace(r"[^\uAC00-\uD7A30-9a-zA-Z\s]",'')

C:\Users\hslio\AppData\Local\Temp\ipykernel_27768\1046961940.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['title'] = df['title'].str.replace(r"[^\uAC00-\uD7A30-9a-zA-Z\s]",'')
C:\Users\hslio\AppData\Local\Temp\ipykernel_27768\1046961940.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace(r"[^\uAC00-\uD7A30-9a-zA-Z\s]",'')


In [27]:
df

,title,content
date,,
2020-09-04,제목 오아시스\n 팬데믹시대에 내마음의 표준을 바꾸는 유통,이번 코로나가 유통시장의 판도를 5년이상 앞당겨진 변화를 가져 왔다매장에서 인테넷의...
2021-04-17,제목 오아시스 오아시스 매수 했습니다,오아이스 주식 중 pe가 보유한 물량이 시장에서 움직이는 듯 하네요어제 아시는 분...
2021-04-17,제목 오아시스 오아시스 매수 했습니다,얼마에 사셨어요 저도 관심이 있는데요
2021-04-17,제목 오아시스\n 오아시스 주식이요,오아시스 주식을 사고싶어게시판에 올려봅니다파실분 계시면 팔육일팔 삼육사일로연락주세요
2021-04-19,제목 오아시스 70만원이면 비싼건가요,30주 70만원에 살려고하는데 비싼건가요
...,...,...
2022-08-31,제목 오아시스 우크라이나와 러시아,덩치 큰 러시아가 누구다 다 이길거라고 했지만 결과는 길고 짧은건 대봐야 안다고우크...
2022-08-31,제목 오아시스 오늘 날짜20220831 더벨 기사 오아시스 기업가치 최대 2조 제시,코스닥 상장을 추진 중인 신선식품 새벽배송 기업 오아시스가 기업가치 목표를 최대 2...
2022-09-02,제목 오아시스 이 시기에 오히려 주목해야 할 기업,연일 금리가 급등하고 있다 매출만을 키우기 위해 무한확장을 거듭했던 스타트업과 유니...


## 토큰화

In [28]:
okt = Okt()

In [29]:
# 불용어 처리 

stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','\n','\n ', '제목']


In [30]:
token = []
for sentence in tqdm(df['title']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    token.append(stopwords_removed_sentence)
    

100%|███████████████████████████████████████████████████████████████████████████████| 136/136 [00:00<00:00, 609.87it/s]


In [46]:
len(token)

136

In [42]:
from collections import Counter

In [51]:
# 한 날짜? 한 행당 횟수 추출

for i in range(136):
    vocab = Counter(token[i])
    print(vocab)

Counter({'오아시스': 1, '팬데믹': 1, '시대': 1, '내': 1, '마음': 1, '표준': 1, '을': 1, '바꾸다': 1, '유통': 1})
Counter({'오아시스': 2, '매수': 1})
Counter({'오아시스': 2, '매수': 1})
Counter({'오아시스': 2, '주': 1, '식이': 1, '요': 1})
Counter({'오아시스': 1, '70만원': 1, '이면': 1, '비싸다': 1, '가요': 1})
Counter({'오아시스': 1, '70만원': 1, '이면': 1, '비싸다': 1, '가요': 1})
Counter({'코멘트': 1, '오아시스': 1, 'pe': 1, '가다': 1, '뭐': 1, '에요': 1})
Counter({'오아시스': 1, '망설이다': 1, '못': 1, '사다': 1})
Counter({'오아시스': 1, '위': 1, '글': 1, '바이럴': 1, '기': 1, '막히다': 1})
Counter({'오아시스': 2, '아직': 1, '모르다': 1, '때': 1, '빨리': 1, '사다': 1})
Counter({'비싸다': 2, '오아시스': 1, '너무': 1})
Counter({'코멘트': 1, '오아시스': 1, '그래도': 1, '지금': 1, '제일': 1, '싸다': 1, '겁니다': 1})
Counter({'비싸다': 2, '오아시스': 1, '너무': 1})
Counter({'오아시스': 2, '상장': 1, '언제': 1, '하나요': 1})
Counter({'오아시스': 1, '옆': 1, '집': 1, '수수': 1, '수지': 1, '그렇게': 1, '일': 1, '없다': 1})
Counter({'오아시스': 1, '거래': 1, '뜸': 1})
Counter({'코멘트': 1, '오아시스': 1, '그': 1, '놈': 1, '싸다': 1, '올리다': 1, '함': 1, '사보다': 1, '거래': 1})
Counter({'오아시스'

In [53]:
# 전체 중 빈도수 출력

all_word_list = sum(token, [])
vocab = Counter(all_word_list)
print(vocab)

Counter({'오아시스': 156, '코멘트': 32, '상장': 12, '주': 11, '일': 11, '되다': 9, '기': 8, '1조': 8, '청구': 8, '비싸다': 7, '액면': 7, '분할': 7, '오다': 7, '총': 7, '액': 7, '가다': 6, '사다': 6, '없다': 6, '가치': 6, '평': 6, '분': 6, '을': 5, '요': 5, '지금': 5, '거래': 5, '실적': 5, '보다': 5, '주식': 5, '기업': 5, '2조': 5, '매수': 4, '너무': 4, '하나요': 4, '가격': 4, '있다': 4, '지다': 4, '3': 4, '가요': 3, '언제': 3, '옆': 3, '집': 3, '수수': 3, '수지': 3, '그렇게': 3, '올리다': 3, '유상증자': 3, '공시': 3, '네': 3, '나오다': 3, '로': 3, '에서': 3, '투자': 3, '반기': 3, '후': 3, '신': 3, '신청': 3, '드리다': 3, '지': 3, '죠': 3, '컬리': 3, '내용': 3, '날짜': 3, '기준': 3, '및': 3, '70만원': 2, '이면': 2, '뭐': 2, '못': 2, '글': 2, '싸다': 2, '겁니다': 2, '그': 2, '놈': 2, '무상증자': 2, '알다': 2, '1': 2, '백만원': 2, '오늘': 2, '예상': 2, '년': 2, '안': 2, '임': 2, '이미': 2, '다르다': 2, '나': 2, '제대로': 2, '희망': 2, '무증': 2, 'ipo': 2, '3조': 2, '4': 2, '대': 2, '괜찮다': 2, '시': 2, '어디서': 2, '확인': 2, '빠르다': 2, '고객': 2, '센터': 2, '악의': 2, '적': 2, '매물': 2, '신고': 2, '나다': 2, '수가': 2, '적다': 2, '그렇다': 2, '하': 2, '않다': 2, '바로': 2, '다': 

In [54]:
word_list = list(vocab.keys())
word_list

['오아시스',
 '팬데믹',
 '시대',
 '내',
 '마음',
 '표준',
 '을',
 '바꾸다',
 '유통',
 '매수',
 '주',
 '식이',
 '요',
 '70만원',
 '이면',
 '비싸다',
 '가요',
 '코멘트',
 'pe',
 '가다',
 '뭐',
 '에요',
 '망설이다',
 '못',
 '사다',
 '위',
 '글',
 '바이럴',
 '기',
 '막히다',
 '아직',
 '모르다',
 '때',
 '빨리',
 '너무',
 '그래도',
 '지금',
 '제일',
 '싸다',
 '겁니다',
 '상장',
 '언제',
 '하나요',
 '옆',
 '집',
 '수수',
 '수지',
 '그렇게',
 '일',
 '없다',
 '거래',
 '뜸',
 '그',
 '놈',
 '올리다',
 '함',
 '사보다',
 '무상증자',
 '알다',
 '1',
 '백만원',
 '되다',
 '매다',
 '오늘',
 '중소기업',
 '여부',
 '예상',
 '말고',
 '올해',
 '실적',
 '보다',
 '2400억',
 '정도인',
 '71',
 '크다',
 '판돈',
 '일부',
 '자다',
 '갈다',
 '년',
 '매출',
 '3천억',
 '안',
 '영',
 '억',
 '이익',
 '100억',
 '남짓',
 '항상',
 '가장',
 '저렴하다',
 '임',
 '유상증자',
 '공시',
 '네',
 '나오다',
 '노력',
 '무조건',
 '회',
 '이미',
 '물량많',
 '금액',
 '실',
 '137만',
 '가격',
 '다르다',
 '나',
 '제대로',
 '성공하다',
 '조용하다',
 '액면',
 '분할',
 '언제나',
 '희망',
 '먹다',
 'K',
 '삼촌',
 '있다',
 '한데',
 '스피드',
 '일단',
 '200',
 '찍다',
 '난후',
 '무증',
 '얘기',
 '14677',
 '1조',
 '평가',
 '로',
 '쏘다',
 '아시아',
 '경제',
 '이다',
 'ib',
 '에서',
 'ipo',
 '가치',
 '5천',
 '

In [55]:
word_rank = list(vocab.values())
word_rank

[156,
 1,
 1,
 1,
 1,
 1,
 5,
 1,
 1,
 4,
 11,
 1,
 5,
 2,
 2,
 7,
 3,
 32,
 1,
 6,
 2,
 1,
 1,
 2,
 6,
 1,
 2,
 1,
 8,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 5,
 1,
 2,
 2,
 12,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 11,
 6,
 5,
 1,
 2,
 2,
 3,
 1,
 1,
 2,
 2,
 2,
 2,
 9,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 5,
 5,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 3,
 3,
 3,
 3,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 4,
 2,
 2,
 2,
 1,
 1,
 7,
 7,
 1,
 2,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 8,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 3,
 2,
 6,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 7,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 3,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 7,
 3,
 6,
 1,
 1,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 3,
 7,
 1,
 3,
 1,
 8,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 6,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 5,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 5,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 1

In [61]:
data = {'word': word_list, 'count' : word_rank}
word_data = pd.DataFrame.from_dict(data)
word_data

,word,count
0,오아시스,156
1,팬데믹,1
2,시대,1
3,내,1
4,마음,1
...,...,...
383,받다,1
384,미트,1
385,박스,1
386,흑자,1


In [68]:
word_data.sort_values(by=['count'], ascending=False).reset_index().drop(columns=['index'])

,word,count
0,오아시스,156
1,코멘트,32
2,상장,12
3,주,11
4,일,11
...,...,...
383,주다,1
384,넘기다,1
385,분과,1
386,식수,1
